In [133]:
import pandas as pd

In [134]:
gdelt = pd.read_parquet('../cache/gdelt.parquet')

In [135]:
gdelt.head()

,Date,Actor1Country,Actor1GeoCountry,Actor1Type,Actor2Country,Actor2GeoCountry,Actor2Type,ActionCountry,EventRootCode,QuadClass,GoldsteinScale,NumSources,NumArticles,AvgTone,Source
0,2019-01-01,None,None,CVL,None,None,None,None,05,1,3.4,4,50,2.354384,https://telegrafi.com/ne-shenj-proteste-labino...
1,2019-01-01,MDV,US,None,USA,US,None,US,19,4,-10.0,2,12,-4.195804,http://www.wgow.com/news/manhunt-on-in-texas-f...
2,2019-01-01,None,US,GOV,None,US,GOV,US,02,1,3.2,10,20,1.734061,https://hanfordsentinel.com/news/national/govt...
3,2019-01-01,RUS,RS,None,None,RS,GOV,RS,06,2,6.0,3,30,-0.427182,http://www.gp.se/nyheter/v%C3%A4rlden/sju-d%C3...
4,2019-01-01,USA,US,None,None,None,None,US,02,1,3.0,8,23,-1.376241,http://www.790wpic.com/news/a-familys-whirlpoo...


In [136]:
gdelt.columns

Index(['Date', 'Actor1Country', 'Actor1GeoCountry', 'Actor1Type',
       'Actor2Country', 'Actor2GeoCountry', 'Actor2Type', 'ActionCountry',
       'EventRootCode', 'QuadClass', 'GoldsteinScale', 'NumSources',
       'NumArticles', 'AvgTone', 'Source'],
      dtype='object')

In [137]:
# convert categorical columns to category type
gdelt.Actor1Country = gdelt.Actor1Country.astype('category')
gdelt.Actor1GeoCountry = gdelt.Actor1GeoCountry.astype('category')
gdelt.Actor1Type = gdelt.Actor1Type.astype('category')
gdelt.Actor2Country = gdelt.Actor2Country.astype('category')
gdelt.Actor2GeoCountry = gdelt.Actor2GeoCountry.astype('category')
gdelt.Actor2Type = gdelt.Actor2Type.astype('category')
gdelt.ActionCountry = gdelt.ActionCountry.astype('category')
gdelt.EventRootCode = gdelt.EventRootCode.astype('category')
gdelt.QuadClass = gdelt.QuadClass.astype('category')

In [138]:
# datetime to unix timestamp
gdelt['Timestamp'] = gdelt.Date.astype('int64') // 10**9

In [139]:
gdelt.dtypes

Date                datetime64[ns]
Actor1Country             category
Actor1GeoCountry          category
Actor1Type                category
Actor2Country             category
Actor2GeoCountry          category
Actor2Type                category
ActionCountry             category
EventRootCode             category
QuadClass                 category
GoldsteinScale             float64
NumSources                   int64
NumArticles                  int64
AvgTone                    float64
Source                      object
Timestamp                    int64
dtype: object

In [140]:
# replace quadclass nums with meaningful names
gdelt['QuadClass'] = gdelt['QuadClass'].map({
    1: 'VerbalCoop', 
    2: 'MaterialCoop', 
    3: 'VerbalConf', 
    4: 'MaterialConf'
})

In [141]:
# replace event root code nums with meaningful names and rename to EventType

# first convert to numeric
gdelt['EventRootCode'] = pd.to_numeric(gdelt['EventRootCode'], errors='coerce')
gdelt['EventRootCode'] = gdelt['EventRootCode'].map({
    1: 'Statement',
    2: 'Appeal',
    3: 'Intent to Coop',
    4: 'Consult',
    5: 'Diplom Coop',
    6: 'Material Coop',
    7: 'Aid',
    8: 'Yield',
    9: 'Investigate',
    10: 'Demand',
    11: 'Disapprove',
    12: 'Reject',
    13: 'Threaten',
    14: 'Protest',
    15: 'Demonstrate Force',
    16: 'Reduce Relations',
    17: 'Coerce',
    18: 'Assault',
    19: 'Fight',
    20: 'Mass Violence'
})
gdelt.rename(columns={'EventRootCode': 'EventType'}, inplace=True)

# convert back to category
gdelt.EventType = gdelt.EventType.astype('category')

In [142]:
# there are 3 NaN values in EventType, so we can drop them
gdelt['EventType'].value_counts(dropna=False)[::-1]

EventType
NaN                        3
Mass Violence           1609
Demonstrate Force      20461
Protest                59635
Reduce Relations       60881
Demand                 69028
Threaten               80823
Assault                81196
Material Coop         106830
Investigate           136693
Reject                146954
Aid                   174983
Yield                 202861
Coerce                255777
Disapprove            305588
Fight                 345688
Intent to Coop        368722
Appeal                418795
Diplom Coop           419667
Statement             648289
Consult              1267084
Name: count, dtype: int64

In [143]:
gdelt.dropna(subset=['EventType'], inplace=True)

In [144]:
# generalize actor types
actor_types = {
    'GOV': ['GOV', 'COP', 'MIL', 'JUD', 'SPY', 'ELI', 'LEG'],
    'Opposition': ['OPP', 'INS', 'REB', 'SEP', 'CRM'],
    'Business': ['BUS', 'AGR', 'DEV', 'LAB'],
    'Benevolent': ['EDU', 'MED', 'HRI', 'HLH', 'ENV'],
}

def generalize_actor_type(actor_type):
    for k, v in actor_types.items():
        if actor_type in v:
            return k
    return 'Other'

In [145]:
gdelt['Actor1TypeGeneral'] = gdelt['Actor1Type'].apply(generalize_actor_type)
gdelt['Actor2TypeGeneral'] = gdelt['Actor2Type'].apply(generalize_actor_type)

In [146]:
gdelt['Actor1TypeGeneral'].value_counts()

Actor1TypeGeneral
GOV           1582931
Other          364790
Benevolent     338947
Business       291860
Opposition     125044
Name: count, dtype: int64

In [147]:
gdelt['Actor2TypeGeneral'].value_counts()

Actor2TypeGeneral
GOV           972947
Other         293756
Benevolent    258098
Business      205176
Opposition     99977
Name: count, dtype: int64

In [148]:
# we can see that GoldsteinScale has 23 NaN values
# this is a very small number, so we can drop them
gdelt[gdelt['GoldsteinScale'].isna()].__len__()

23

In [149]:
gdelt.dropna(subset=['GoldsteinScale'], inplace=True)

In [150]:
# replace countries not in the top 20 with 'Other'
top_countries = gdelt['Actor1Country'].value_counts().head(20).index
gdelt['Actor1Country'] = gdelt['Actor1Country'].apply(lambda x: x if x in top_countries else 'Other')
gdelt['Actor2Country'] = gdelt['Actor2Country'].apply(lambda x: x if x in top_countries else 'Other')

In [151]:
# replace geo countries not in the top 20 with 'Other'
top_geo_countries = gdelt['Actor1GeoCountry'].value_counts().head(20).index
gdelt['Actor1GeoCountry'] = gdelt['Actor1GeoCountry'].apply(lambda x: x if x in top_geo_countries else 'Other')
gdelt['Actor2GeoCountry'] = gdelt['Actor2GeoCountry'].apply(lambda x: x if x in top_geo_countries else 'Other')

In [152]:
# replace action countries not in the top 20 with 'Other'
top_action_countries = gdelt['ActionCountry'].value_counts().head(20).index
gdelt['ActionCountry'] = gdelt['ActionCountry'].apply(lambda x: x if x in top_action_countries else 'Other')

In [153]:
gdelt[gdelt.NumArticles > 500].shape[0]

99

There are only 99 events with more than 500 articles, so we can cap the number of articles at 500 to avoid outliers.

In [154]:
# cap number of articles at 500
gdelt['NumArticles'] = gdelt['NumArticles'].clip(upper=500)

In [155]:
# reorder columns
gdelt = gdelt[['Date', 'Timestamp',  # Date
    'Actor1Country', 'Actor1GeoCountry', 'Actor1Type', 'Actor1TypeGeneral',  # Actor1
    'Actor2Country', 'Actor2GeoCountry', 'Actor2Type', 'Actor2TypeGeneral',  # Actor2
    'ActionCountry', 'EventType', 'QuadClass', 'GoldsteinScale',  # Event
    'NumSources', 'NumArticles', 'AvgTone', 'Source']]  # Reaction

In [156]:
from funcs.advanced_describe import advanced_describe

advanced_describe(gdelt, drop_cols=['Date', 'Timestamp', 'Source'])

,present,unique,top,share,mean,min,50%,max
Actor1Country,44%,21,Other,13.97%,NaN,NaN,NaN,NaN
Actor1GeoCountry,83%,21,Other,26.21%,NaN,NaN,NaN,NaN
Actor1Type,52%,32,GOV,16.47%,NaN,NaN,NaN,NaN
Actor1TypeGeneral,52%,5,GOV,30.61%,NaN,NaN,NaN,NaN
Actor2Country,32%,21,Other,10.58%,NaN,NaN,NaN,NaN
Actor2GeoCountry,58%,21,Other,18.63%,NaN,NaN,NaN,NaN
Actor2Type,35%,32,GOV,10.15%,NaN,NaN,NaN,NaN
Actor2TypeGeneral,35%,5,GOV,18.81%,NaN,NaN,NaN,NaN
ActionCountry,94%,21,Other,29.85%,NaN,NaN,NaN,NaN
EventType,100%,20,Consult,24.50%,NaN,NaN,NaN,NaN


In [157]:
gdelt.head()

,Date,Timestamp,Actor1Country,Actor1GeoCountry,Actor1Type,Actor1TypeGeneral,Actor2Country,Actor2GeoCountry,Actor2Type,Actor2TypeGeneral,ActionCountry,EventType,QuadClass,GoldsteinScale,NumSources,NumArticles,AvgTone,Source
0,2019-01-01,1546300800,NaN,NaN,CVL,Other,NaN,NaN,NaN,NaN,NaN,Diplom Coop,VerbalCoop,3.4,4,50,2.354384,https://telegrafi.com/ne-shenj-proteste-labino...
1,2019-01-01,1546300800,Other,US,NaN,NaN,USA,US,NaN,NaN,US,Fight,MaterialConf,-10.0,2,12,-4.195804,http://www.wgow.com/news/manhunt-on-in-texas-f...
2,2019-01-01,1546300800,NaN,US,GOV,GOV,NaN,US,GOV,GOV,US,Appeal,VerbalCoop,3.2,10,20,1.734061,https://hanfordsentinel.com/news/national/govt...
3,2019-01-01,1546300800,RUS,RS,NaN,NaN,NaN,RS,GOV,GOV,RS,Material Coop,MaterialCoop,6.0,3,30,-0.427182,http://www.gp.se/nyheter/v%C3%A4rlden/sju-d%C3...
4,2019-01-01,1546300800,USA,US,NaN,NaN,NaN,NaN,NaN,NaN,US,Appeal,VerbalCoop,3.0,8,23,-1.376241,http://www.790wpic.com/news/a-familys-whirlpoo...


In [158]:
# save cleaned gdelt
gdelt.to_parquet('../cache/gdelt_cleaned.parquet')